In [26]:
import pandas as pd
import warnings
import nltk
import numpy as np

warnings.filterwarnings('ignore')

In [27]:
# Download necessary NLTK data (only needs to be done once)
try:
    nltk.data.find('tokenizers/punkt')
    print("NLTK 'punkt' tokenizer data found.")
except LookupError: # Use LookupError, which is raised when the resource isn't found
    print("NLTK 'punkt' tokenizer data not found. Downloading...")
    nltk.download('punkt')

NLTK 'punkt' tokenizer data found.


In [28]:
df = pd.read_csv('ramayana_shloka_dataset.csv')


In [29]:
verses = df['translation'].tolist()
len(verses)

18305

In [30]:
verses

["The ascetic Valmiki inquired Naarada - 'who engaged deeply in austerity and study of Vedas, best among the eloquent and eminent among sages' (as follows:).",
 '"Who really is that person in this present world, who is virtuous and vigorous, a conscientious \n\tone, one who is mindful of good deeds done to him, and also a speaker of truth and who is determined in his deed...',
 '"Who is appropriate in disposition... who is interested in welfare of all beings... who is \n\tadept and also an able one... also uniquely pleasant to look at...',
 '"Who is that self-composed one, who controlled his ire, who is brilliant, non-jealous and whom do even\n\tthe gods fear, when provoked to war...',
 '"All this I wish to listen from you, oh! Great Sage, you are competent to know this kind of man, \nand indeed my inquisitiveness is immense..." Thus Valmiki enquired with Narada.',
 'On listening all those words of Valmiki, Narada, the knower of all the three worlds, said "let \nit be heard..." and bec

In [31]:
from nltk import word_tokenize
import nltk

In [32]:
# You might need to download the 'punkt' resource if you haven't already
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

In [33]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\parth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [34]:
verses[0]

"The ascetic Valmiki inquired Naarada - 'who engaged deeply in austerity and study of Vedas, best among the eloquent and eminent among sages' (as follows:)."

## creating sentence embeddings

In [ ]:
# Example using the original verses list
from sentence_transformers import SentenceTransformer

# Assuming 'verses' is your list of original verse strings
# Perform minimal cleaning if necessary (e.g., remove NaN if they weren't handled before)
original_verses_cleaned = [str(v) for v in verses if isinstance(v, str)]

# Load the model
model_name = 'all-mpnet-base-v2' # Higher quality, slower
# model_name = 'all-MiniLM-L6-v2' # Good balance, faster
model = SentenceTransformer(model_name)

print(f"Encoding {len(original_verses_cleaned)} original verses using {model_name}...")

# Encode the original verses
verse_embeddings = model.encode(original_verses_cleaned, show_progress_bar=True)

print(f"Created embeddings of shape: {verse_embeddings.shape}")

# Now verse_embeddings[i] corresponds to the embedding of original_verses_cleaned[i]


Encoding 18303 original verses using all-mpnet-base-v2...


Batches:   0%|          | 0/572 [00:00<?, ?it/s]

Created embeddings of shape: (18303, 768)


In [36]:
import numpy as np

# Assuming verse_embeddings is your NumPy array
np.save('verse_embeddings.npy', verse_embeddings)



In [37]:

# To load it back later:
loaded_embeddings = np.load('verse_embeddings.npy')

In [38]:
loaded_embeddings

array([[ 0.05972938,  0.03558729, -0.03178061, ...,  0.02474446,
        -0.01520113, -0.05065229],
       [ 0.0663133 ,  0.02948586,  0.02220144, ...,  0.01881714,
         0.03869649, -0.01806053],
       [ 0.04414972,  0.03080636,  0.00757363, ...,  0.02731661,
         0.04218174, -0.0087631 ],
       ...,
       [ 0.0697385 ,  0.07671425, -0.03781043, ...,  0.01275892,
        -0.04242986, -0.06457641],
       [ 0.02225219,  0.04885808, -0.02248592, ..., -0.0048963 ,
        -0.00545983, -0.06565246],
       [ 0.05690006,  0.02885873,  0.00602078, ...,  0.00255241,
         0.02866167, -0.06798723]], shape=(18303, 768), dtype=float32)

In [39]:
# Add these to your imports cell or a new cell near the top
import torch
from sentence_transformers import util
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np
import pandas as pd # Make sure pandas is imported for the table


In [40]:

if 'verse_embeddings' in locals() and isinstance(verse_embeddings, np.ndarray):
    verse_embeddings_tensor = torch.tensor(verse_embeddings)
    print("Converted verse_embeddings NumPy array to PyTorch tensor.")
else:
    print("Error: 'verse_embeddings' numpy array not found. Please run the embedding creation cell first.")
    # Add placeholder or raise error if needed
    verse_embeddings_tensor = torch.empty(0)



Converted verse_embeddings NumPy array to PyTorch tensor.


In [41]:

# Ensure original_verses_cleaned exists from the previous cell
if 'original_verses_cleaned' not in locals():
     print("Error: 'original_verses_cleaned' list not found. Please run the embedding creation cell first.")
     # Add placeholder or raise error if needed
     original_verses_cleaned = []


In [42]:

# Ensure the sentence transformer model ('model') exists from the previous cell
if 'model' not in locals():
    print("Error: Sentence Transformer 'model' not found. Please run the embedding creation cell first.")
    # Add placeholder or raise error if needed
    embedding_model_placeholder = None # Use a placeholder name if 'model' is None
else:
    embedding_model_placeholder = model # Use the actual model



In [73]:

def is_relevant(statement: str, embedding_model, context_embeddings_tensor, threshold: float = 0.35) -> bool:
    """Checks if a statement is semantically relevant to the Ramayana verses."""
    if not statement or embedding_model is None or context_embeddings_tensor.numel() == 0:
        print("Relevance check skipped: Missing statement, model, or embeddings.")
        return False
    try:
        # Encode the statement and ensure it's a tensor
        stmt_emb = torch.tensor(embedding_model.encode(statement))
        # Ensure stmt_emb is 2D for cos_sim: [1, embedding_dim]
        if stmt_emb.ndim == 1:
            stmt_emb = stmt_emb.unsqueeze(0)

        # Compute cosine similarity
        # Move tensors to the same device if necessary (e.g., GPU)
        if torch.cuda.is_available() and not stmt_emb.is_cuda:
            stmt_emb = stmt_emb.to('cuda')
        if torch.cuda.is_available() and not context_embeddings_tensor.is_cuda:
             # Move only if not already on GPU to avoid errors
             if not context_embeddings_tensor.is_cuda:
                 context_embeddings_tensor = context_embeddings_tensor.to('cuda')


        cos_scores = util.cos_sim(stmt_emb, context_embeddings_tensor)[0] # Get scores for the single statement

        # Find the maximum similarity score
        max_score = torch.max(cos_scores).item() # .item() gets the Python float

        print(f"Relevance check: Max similarity score = {max_score:.4f}")
        return max_score >= threshold
    except Exception as e:
        print(f"Error during relevance check: {e}")
        return False


In [74]:

def retrieve_context(statement: str, embedding_model, context_embeddings_tensor, original_verses: list, top_n: int = 5) -> list[str]:
    """Retrieves the top_n most similar verses to the statement."""
    if not statement or embedding_model is None or context_embeddings_tensor.numel() == 0 or not original_verses:
        print("Context retrieval skipped: Missing statement, model, embeddings, or original verses.")
        return []
    try:
        stmt_emb = torch.tensor(embedding_model.encode(statement))
        if stmt_emb.ndim == 1:
            stmt_emb = stmt_emb.unsqueeze(0)

        # Move tensors to the same device if necessary
        if torch.cuda.is_available() and not stmt_emb.is_cuda:
            stmt_emb = stmt_emb.to('cuda')
        if torch.cuda.is_available():
             if not context_embeddings_tensor.is_cuda:
                 context_embeddings_tensor = context_embeddings_tensor.to('cuda')


        # Compute cosine similarity
        cos_scores = util.cos_sim(stmt_emb, context_embeddings_tensor)[0]

        # Get top N scores and their indices
        actual_top_n = min(top_n, len(original_verses))
        # Ensure k is not greater than the number of available scores
        if actual_top_n > cos_scores.shape[0]:
             actual_top_n = cos_scores.shape[0]

        if actual_top_n <= 0:
            print("No context available to retrieve.")
            return []

        top_results = torch.topk(cos_scores, k=actual_top_n)

        # Retrieve the corresponding verses
        top_indices = top_results.indices.cpu().tolist() # Move indices to CPU before converting to list
        retrieved_verses = [original_verses[i] for i in top_indices]
        print(f"Retrieved {len(retrieved_verses)} verses for context.")
        return retrieved_verses
    except Exception as e:
        print(f"Error during context retrieval: {e}")
        return []


In [75]:

verifier_model_name = "google/flan-t5-large"
print(f"Loading LLM for verification: {verifier_model_name}...")


Loading LLM for verification: google/flan-t5-large...


In [76]:

# Determine device
device = 0 if torch.cuda.is_available() else -1
device_name = "GPU"
# if device == 0 else "CPU"
print(f"Attempting to load model on: {device_name}")


Attempting to load model on: GPU


In [82]:
# Load tokenizer and model explicitly
tokenizer = AutoTokenizer.from_pretrained(verifier_model_name)
model_llm = AutoModelForSeq2SeqLM.from_pretrained(verifier_model_name) # Renamed to model_llm

# Move model to the determined device
if device == 0:
    model_llm.to('cuda') # Move model to GPU if available

# Create the pipeline using the loaded model and tokenizer, specifying the device
verifier_pipeline = pipeline(
    "text2text-generation",
    model=model_llm,
    tokenizer=tokenizer,
    device=device # Explicitly set device for pipeline
    )
print(f"LLM loaded successfully on {device_name}.")




# Define the few-shot prompt template (Modified slightly for UNKNOWN)
few_shot_template = """
Based *only* on the provided Context, determine if the Statement is TRUE or FALSE. If the context does not contain enough information to confirm or deny the statement, answer UNKNOWN. Output only the single word TRUE, FALSE, or UNKNOWN.

Context: The ascetic Valmiki inquired Naarada - 'who engaged deeply in austerity and study of Vedas, best among the eloquent and eminent among sages' (as follows:).
Statement: Valmiki asked Narada a question.
Answer: TRUE

Context: "He is lion-chested, with concealed collarbones, knee-length are his arms, and his is a long bow, an enemy-subjugator, \\\\nand his emperor\\\\'s countenance is with a crowning-head and an ample forehead, and his gait is majestic like that of a lion..."
Statement: Rama had short arms.
Answer: FALSE

Context: "That city is surrounded with gateways and archways; the front yards of buildings are well \\\\nlaid; it is lodges all kinds of machinery, weaponry and craftsmen, and king Dasharatha dwells \\\\nin such a city."
Statement: Ayodhya was a small village.
Answer: FALSE

Context: "Indra procreated the lord of vanara-s, namely Vali, who by his physique is like \\\\n\\\\t\\\\t\\\\tMt. Mahendra, and the highest humidifier among all the humidifiers, namely the \\\\n\\\\t\\\\t\\\\tSun, procreated Sugreeva."
Statement: Hanuman was the son of Indra.
Answer: FALSE

Context: "The Sun-god will not verily parch Ravana, the Air-god will not puff at his sides, and on \\\\nseeing Ravana, the lord of the undulating waves, namely the ocean will not pulsate either."
Statement: Ravana commanded the elements like Sun and Wind.
Answer: TRUE

Context: {context}
Statement: {stmt}
Answer:
""" 



Device set to use cpu


LLM loaded successfully on GPU.


In [87]:
# Verification Function using the loaded pipeline
def verify_with_flan_t5(statement: str, context_verses: list) -> bool | None | str:
    """Verifies the statement against the context using FLAN-T5 with few-shot prompting."""
    if not verifier_pipeline or not context_verses:
        print("Verifier pipeline not loaded or no context provided.")
        return None # Indicate failure or inability to verify

    context_str = "\n".join(context_verses)
    # Limit context length - FLAN-T5 large still has limits (often 512 tokens total input)
    max_context_tokens = 1026 # Conservative limit
    try:
        # Use the tokenizer associated with the pipeline for consistency
        context_tokens = verifier_pipeline.tokenizer.encode(context_str, max_length=max_context_tokens, truncation=True)
        context_str = verifier_pipeline.tokenizer.decode(context_tokens, skip_special_tokens=True)
    except Exception as e:
        print(f"Error during context tokenization/truncation: {e}")
        return None # Cannot proceed without valid context

    prompt = few_shot_template.format(stmt=statement, context=context_str)

    try:
        # Adjust generation parameters
        output = verifier_pipeline(
            prompt,
            max_new_tokens=10, # Short answer expected
            num_beams=3,
            early_stopping=True,
            temperature=0 # Deterministic output
            )
        result_text = output[0]['generated_text'].strip().upper()
        print(f"FLAN-T5 Raw Output: '{result_text}'")

        # Robust checking
        if result_text.startswith("TRUE"):
            return True
        elif result_text.startswith("FALSE"):
            return False
        elif result_text.startswith("UNKNOWN"):
            return "UNKNOWN"
        else:
            # Fallback check if startswith fails
            if "TRUE" in result_text: return True
            if "FALSE" in result_text: return False
            if "UNKNOWN" in result_text: return "UNKNOWN"

            print(f"Warning: Model output '{result_text}' not clearly TRUE, FALSE, or UNKNOWN.")
            return None # Cannot determine
    except Exception as e:
        print(f"Error during verification pipeline call: {e}")
        if "CUDA out of memory" in str(e):
            print("CUDA out of memory error. Consider using a smaller model or reducing context length.")
        return None


In [88]:
# --- Main Fact-Checking Function (Modified for None Output) ---

def check_ramayana_statement(statement: str,
                             embedding_model, # Sentence Transformer model
                             context_embeddings_tensor, # Tensor of verse embeddings
                             original_verses: list, # List of original verse strings
                             relevance_threshold: float = 0.35,
                             top_n_context: int = 5):
    """
    Checks if a statement about the Ramayana is TRUE or FALSE. Returns None if
    the statement is irrelevant, cannot be verified (UNKNOWN), or if an error occurs.

    Args:
        statement: The statement to check.
        embedding_model: The loaded SentenceTransformer model.
        context_embeddings_tensor: PyTorch tensor of verse embeddings.
        original_verses: List of the original verse text strings.
        relevance_threshold: Cosine similarity threshold for relevance.
        top_n_context: Number of top verses to retrieve for context.

    Returns:
        - True: If the statement is relevant and verified as true.
        - False: If the statement is relevant and verified as false.
        - None: If the statement is irrelevant, unknown, or an error occurred.
    """
    print(f"\n--- Checking Statement: '{statement}' ---")

    # 1. Relevance Check
    relevant = is_relevant(statement, embedding_model_placeholder, context_embeddings_tensor, relevance_threshold)
    if not relevant:
        print("Result: Statement deemed IRRELEVANT. Returning None.")
        # MODIFICATION: Return None for irrelevant statements
        return None

    print("Statement is relevant. Proceeding to context retrieval...")

    # 2. Context Retrieval
    context = retrieve_context(statement, embedding_model_placeholder, context_embeddings_tensor, original_verses, top_n=top_n_context)
    if not context:
        print("Result: Could not retrieve context. Cannot verify. Returning None.")
        return None # Indicate error or inability to retrieve

    print("\nRetrieved Context Verses (Snippets):")
    for i, v in enumerate(context):
        print(f"{i+1}. {v[:150]}...") # Print snippet

    # 3. Verification
    print("\nVerifying statement with FLAN-T5...")
    verification_result = verify_with_flan_t5(statement, context)

    # MODIFICATION: Check for "UNKNOWN" and map it to None
    if verification_result == "UNKNOWN":
        print(f"Final Verification Result: UNKNOWN. Returning None.")
        return None
    elif verification_result is None:
         print(f"Final Verification Result: Error/Ambiguous. Returning None.")
         return None # Already None if verify_with_flan_t5 failed
    else:
        # Only True or False remain
        print(f"Final Verification Result: {verification_result}")
        return verification_result # Return True or False



In [89]:
# --- Test Suite Execution and Formatted Summary (Corrected Logic with Higher Threshold) ---
import pandas as pd # Ensure pandas is imported

# --- Test Data (Expected values for 21, 22 are None) ---
test_data = [
    {"ID": 1, "Statement": "Rama is the eldest son of King Dasharatha.", "Expected": True, "Reference": "Bala Kanda, Sarga 18, Shloka 24"},
    {"ID": 2, "Statement": "Sita was discovered by King Janaka in a furrow during ploughing and was later adopted by him.", "Expected": True, "Reference": "Bala Kanda, Sarga 66, Shloka 14"},
    {"ID": 3, "Statement": "Lakshmana, Rama’s devoted younger brother, accompanied him into exile.", "Expected": True, "Reference": "Ayodhya Kanda, Sarga 31, Shloka 30"},
    {"ID": 4, "Statement": "Bharata, another brother of Rama, revered him and ruled as regent in his absence.", "Expected": True, "Reference": "Ayodhya Kanda, Sarga 115, Shloka 22"},
    {"ID": 5, "Statement": "Hanuman was the best minister of Sugreeva.", "Expected": True, "Reference": "Kishkindha Kanda, Sarga 6, Shloka 2"},
    {"ID": 6, "Statement": "Ravana, the demon king of Lanka, abducted Sita, setting in motion the events of the epic.", "Expected": True, "Reference": "Aranya Kanda, Sarga 49, Shloka 20"},
    {"ID": 7, "Statement": "On seeing Hanuman, Sita lost her consciousness for a long time.", "Expected": True, "Reference": "Sundara Kanda, Sarga 32, Shloka 8"},
    {"ID": 8, "Statement": "Nila, a general of the monkey army, was rendered unconscious by a mystic arrow shot by Ravana.", "Expected": True, "Reference": "Yuddha Kanda, Sarga 59, Shloka 89"},
    {"ID": 9, "Statement": "Nala constructed a bridge for Rama to cross over to Lanka.", "Expected": True, "Reference": "Yuddha Kanda, Sarga 27, Shloka 61"},
    {"ID": 10, "Statement": "The sage Vishwamitra mentored Rama during his early life, imparting knowledge of warfare and ritual.", "Expected": True, "Reference": "Bala Kanda, Sarga 22"},
    {"ID": 11, "Statement": "Sita herself built the bridge to Lanka with the help of the monkey army.", "Expected": False, "Reference": ""},
    {"ID": 12, "Statement": "Lakshmana was a disciple of Ravana and learned warfare techniques from him.", "Expected": False, "Reference": ""},
    {"ID": 13, "Statement": "Bharata was originally from Lanka and was later adopted by King Dasharatha.", "Expected": False, "Reference": ""},
    {"ID": 14, "Statement": "After meeting Sita, Hanuman lost his ability to speak entirely.", "Expected": False, "Reference": ""},
    {"ID": 15, "Statement": "Ravana is depicted solely as a peaceful poet who never engaged in any battles.", "Expected": False, "Reference": ""},
    {"ID": 16, "Statement": "Rama’s exile lasted only one year rather than the traditionally recounted fourteen years.", "Expected": False, "Reference": ""},
    {"ID": 17, "Statement": "Sita and Rama were married in Lanka with Ravana acting as the priest for their wedding.", "Expected": False, "Reference": ""},
    {"ID": 18, "Statement": "Lakshmana is not mentioned at all in the Valmiki Ramayana.", "Expected": False, "Reference": ""},
    {"ID": 19, "Statement": "Bharata built a grand palace in Lanka following Rama’s victory over Ravana.", "Expected": False, "Reference": ""},
    {"ID": 20, "Statement": "Hanuman was the son of Ravana.", "Expected": False, "Reference": ""},
    # Expected is None for irrelevant/unknown
    {"ID": 21, "Statement": "The Ramayana describes the invention of the internet.", "Expected": None, "Reference": ""},
    {"ID": 22, "Statement": "What was the exact color of Rama's favorite chariot?", "Expected": None, "Reference": ""},
]


In [90]:

results_summary = []

print("\n--- Running Test Suite ---")

# Check if pipeline components are ready
pipeline_ready = (
    'original_verses_cleaned' in locals() and
    'embedding_model_placeholder' in locals() and embedding_model_placeholder is not None and
    'verifier_pipeline' in locals() and verifier_pipeline is not None and
    'verse_embeddings_tensor' in locals() and verse_embeddings_tensor.numel() > 0
)

if not pipeline_ready:
    print("\nERROR: Pipeline components (model, embeddings, verifier) not fully initialized. Cannot run tests.")
    print("Please ensure previous cells have executed successfully.")
else:
    # Set default parameters for the check function if needed
    # --- MODIFICATION: Increased threshold ---
    default_relevance_threshold = 0.35 # Increased from 0.35
    # --- END MODIFICATION ---
    default_top_n_context = 10 # Adjust as needed

    print(f"Using Relevance Threshold: {default_relevance_threshold}") # Optional: Confirm threshold used

    for item in test_data:
        statement_id = item["ID"]
        statement_text = item["Statement"]
        expected_result = item["Expected"] # Now expects True, False, or None

        print(f"\n--- Checking Statement {statement_id}: '{statement_text}' ---")

        # Call the main checking function (which now returns None for irrelevant/unknown)
        # It will use the higher default_relevance_threshold set above
        actual_result = check_ramayana_statement(
            statement=statement_text,
            embedding_model=embedding_model_placeholder,
            context_embeddings_tensor=verse_embeddings_tensor,
            original_verses=original_verses_cleaned,
            relevance_threshold=default_relevance_threshold,
            top_n_context=default_top_n_context
        )

        print(f"Expected: {expected_result}, Actual: {actual_result}")

        # Compare actual result with expected result (handles None == None correctly)
        match = actual_result == expected_result

        results_summary.append({
            "ID": statement_id,
            "Statement": statement_text,
            "Expected": expected_result,
            "Actual": actual_result, # Store the actual result (True, False, or None)
            "Match": "Yes" if match else "No" # Match is determined by the comparison above
        })

    print("\n--- Test Suite Summary ---")
    # Define header format
    header = f"{'ID':<3} | {'Expected':<10} | {'Actual':<10} | {'Match':<5} | {'Statement'}"
    print(header)
    print("-" * (len(header) + 20)) # Adjust separator length

    # Print each result row using the stored actual results and match status
    for res in results_summary:
        expected_str = str(res['Expected'])
        actual_str = str(res['Actual']) # Converts the stored result (True, False, None) to string
        match_str = res['Match']        # Uses the calculated match status

        print(f"{res['ID']:<3} | {expected_str:<10} | {actual_str:<10} | {match_str:<5} | {res['Statement']}")

    # Optional: Calculate and print overall accuracy (based on the calculated match status)
    match_count = sum(1 for res in results_summary if res['Match'] == 'Yes')
    total_cases = len(results_summary)
    if total_cases > 0:
        accuracy = match_count / total_cases
        print(f"\nOverall Accuracy: {match_count} / {total_cases} = {accuracy:.2%}")
    else:
        print("\nNo test cases were run.")




--- Running Test Suite ---
Using Relevance Threshold: 0.35

--- Checking Statement 1: 'Rama is the eldest son of King Dasharatha.' ---

--- Checking Statement: 'Rama is the eldest son of King Dasharatha.' ---
Relevance check: Max similarity score = 0.8394
Statement is relevant. Proceeding to context retrieval...
Retrieved 10 verses for context.

Retrieved Context Verses (Snippets):
1. "Rama is the son of king Dasharatha, one born in Ikshvaku dynasty, an adept one in virtue and thus an 
adherent of his father's orders......
2. "There is some king known as Dasharatha, an enhancer of Ikshvaku dynasty, and to him
there will be a great-resplendent a son who will be known as Rama...
3. "He whose son is Raghava, is a resolute king known as Dasharatha. That king is a kingly rampart for 
righteousness, a kingpin abiding in forthrightnes...
4. "Dasharatha's son Rama is the one with inescapable arms and keen-eyed, but has jute-cloths and skin 
of black-deer for his dress, yet in semblance he ...

# Test Suite Summary

| ID  | Expected | Actual  | Match | Statement                                                                                                |
|-----|----------|---------|-------|----------------------------------------------------------------------------------------------------------|
| 1   | True     | True    | Yes   | Rama is the eldest son of King Dasharatha.                                                               |
| 2   | True     | False   | No    | Sita was discovered by King Janaka in a furrow during ploughing and was later adopted by him.             |
| 3   | True     | True    | Yes   | Lakshmana, Rama’s devoted younger brother, accompanied him into exile.                                     |
| 4   | True     | False   | No    | Bharata, another brother of Rama, revered him and ruled as regent in his absence.                         |
| 5   | True     | True    | Yes   | Hanuman was the best minister of Sugreeva.                                                                |
| 6   | True     | True    | Yes   | Ravana, the demon king of Lanka, abducted Sita, setting in motion the events of the epic.                 |
| 7   | True     | True    | Yes   | On seeing Hanuman, Sita lost her consciousness for a long time.                                           |
| 8   | True     | False   | No    | Nila, a general of the monkey army, was rendered unconscious by a mystic arrow shot by Ravana.           |
| 9   | True     | True    | Yes   | Nala constructed a bridge for Rama to cross over to Lanka.                                               |
| 10  | True     | True    | Yes   | The sage Vishwamitra mentored Rama during his early life, imparting knowledge of warfare and ritual.      |
| 11  | False    | False   | Yes   | Sita herself built the bridge to Lanka with the help of the monkey army.                                  |
| 12  | False    | False   | Yes   | Lakshmana was a disciple of Ravana and learned warfare techniques from him.                               |
| 13  | False    | False   | Yes   | Bharata was originally from Lanka and was later adopted by King Dasharatha.                              |
| 14  | False    | False   | Yes   | After meeting Sita, Hanuman lost his ability to speak entirely.                                           |
| 15  | False    | False   | Yes   | Ravana is depicted solely as a peaceful poet who never engaged in any battles.                            |
| 16  | False    | False   | Yes   | Rama’s exile lasted only one year rather than the traditionally recounted fourteen years.                  |
| 17  | False    | False   | Yes   | Sita and Rama were married in Lanka with Ravana acting as the priest for their wedding.                   |
| 18  | False    | False   | Yes   | Lakshmana is not mentioned at all in the Valmiki Ramayana.                                                |
| 19  | False    | True    | No    | Bharata built a grand palace in Lanka following Rama’s victory over Ravana.                               |
| 20  | False    | False   | Yes   | Hanuman was the son of Ravana.                                                                            |
| 21  | None     | False   | No    | The Ramayana describes the invention of the internet.                                                    |
| 22  | None     | True    | No    | What was the exact color of Rama's favorite chariot?                                                     |

**Overall Accuracy: 16 / 22 = 72.73%**